In [2]:
pip install --upgrade fosforml

     |████████████████████████████████| 51kB 5.1MB/s eta 0:00:011
     |████████████████████████████████| 1.9MB 8.4MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 40.7MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 62.8MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 126kB/s s eta 0:00:01                         | 25.7MB 68.6MB/s eta 0:00:03
     |████████████████████████████████| 133kB 103.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 27.0MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 65.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 24.4MB/s eta 0:00:01
     |████████████████████████████████| 98.7MB 308kB/s  eta 0:00:01
     |████████████████████████████████| 51kB 19.5MB/s eta 0:00:01
     |████████████████████████████████| 174kB 102.0MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 63.6MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 62.0MB/s eta 0:00:01
    

In [3]:
import pandas as pd
import numpy as np
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [ ]:
my_session.connection.database

In [ ]:
my_session.connection.schema

In [ ]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"